<a href="https://colab.research.google.com/github/Kylehanna/Instacart-Market-Basket-Analysis/blob/main/03_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os 
from statsmodels.graphics.mosaicplot import mosaic
color = sns.color_palette()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../../data/instacartData_prior.csv')

In [ ]:
order_products_train = pd.read_csv('../../data/instacartData_train.csv')

In [ ]:
products = pd.read_csv('../../data/instacartData_train.csv')

In [ ]:
df.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0
1,1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0
2,2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0
3,3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0
4,4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry,202279,prior,3,5,9,8.0


In [ ]:
#top_users = df.user_id.value_counts().values[:300000]

In [ ]:
#df_temp = df[df['user_id'].isin(top_users)]

In [ ]:
#df_temp.shape

In [ ]:
#df_temp.user_id.nunique()

In [ ]:
#df_temp.info()

In [ ]:
df.describe()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.035642e+07
mean,1.621724e+07,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01,7.121430e+01,9.921906e+00,1.029372e+05,1.714205e+01,2.738818e+00,1.342498e+01,1.110407e+01
std,9.363031e+06,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01,3.820302e+01,6.281156e+00,5.946648e+04,1.753504e+01,2.090049e+00,4.246365e+00,8.778914e+00
min,0.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.108622e+06,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00,5.142100e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00
50%,1.621724e+07,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00,1.026110e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00
75%,2.432587e+07,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01,1.543910e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.243449e+07,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01


In [ ]:
df.eval_set.unique()

array(['prior'], dtype=object)

In [ ]:
df.shape

(32434489, 16)

## Feature Engineering

In [ ]:
#Total number of orders placed by each users
users = df.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()

In [ ]:
users.head()

,user_id,u_num_of_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [ ]:
# Average number of products bought by the user in each purchase.
#1. First getting the total number of products in each order.
total_prd_per_order = df.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()
total_prd_per_order.head(10)

,user_id,order_id,total_products_per_order
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,2254736,5
4,1,2295261,6
5,1,2398795,6
6,1,2539329,5
7,1,2550362,9
8,1,3108588,6
9,1,3367565,4


In [ ]:
#2. Getting the average number of products purchased by each user
avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [ ]:
#deleting the total_prd_per_order dataframe
del total_prd_per_order

In [ ]:
#dow of most orders placed by each user
from scipy import stats
dow = df.groupby(by='user_id')['order_dow'].agg(lambda x: stats.mode(x)[0]).to_frame('dow_most_orders_u').reset_index()
dow.head()

,user_id,dow_most_orders_u
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


In [ ]:
#hour of day when most orders placed by each user
from scipy import stats
hod = df.groupby(by='user_id')['order_hour_of_day'].agg(lambda x: stats.mode(x)[0]).to_frame('hod_most_orders_u').reset_index()
hod.head()

,user_id,hod_most_orders_u
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


In [ ]:
#merging the user created features.

#1. merging avg_products with users
users = users.merge(avg_products, on='user_id', how='left')
#deleting avg_products
del avg_products
users.head()

,user_id,u_num_of_orders,u_avg_prd
0,1,10,5.900000
1,2,14,13.928571
2,3,12,7.333333
3,4,5,3.600000
4,5,4,9.250000


In [ ]:
#2. merging dow with users.
users = users.merge(dow, on='user_id', how='left')
#deleting dow
del dow
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,10,5.900000,4
1,2,14,13.928571,2
2,3,12,7.333333,0
3,4,5,3.600000,4
4,5,4,9.250000,3


In [ ]:
#number of times a product was purchased.
prd = df.groupby(by='product_id')['order_id'].agg('count').to_frame('prd_count_p').reset_index()
prd.head()

,product_id,prd_count_p
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [ ]:
#products reorder ratio.
reorder_p = df.groupby(by='product_id')['reordered'].agg('mean').to_frame('p_reordered_ratio').reset_index()
reorder_p.head()

,product_id,p_reordered_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [ ]:
#merging the reorder_p with prd
prd = prd.merge(reorder_p, on='product_id', how='left')
#deleting reorder_p
del reorder_p
prd.head()

,product_id,prd_count_p,p_reordered_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.600000


In [ ]:
#how many times has a user bought the same product?
uxp = df.groupby(by=['user_id', 'product_id'])['order_id'].agg('count').to_frame('uxp_times_bought').reset_index()
uxp.head()

,user_id,product_id,uxp_times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [ ]:
#reorder ratio of the user for each product.
reorder_uxp = df.groupby(by=['user_id', 'product_id'])['reordered'].agg('mean').to_frame('uxp_reordered_ratio').reset_index()
reorder_uxp.head()

,user_id,product_id,uxp_reordered_ratio
0,1,196,0.900000
1,1,10258,0.888889
2,1,10326,0.000000
3,1,12427,0.900000
4,1,13032,0.666667


In [ ]:
#merging the two dataframes into one
uxp = uxp.merge(reorder_uxp, on=['user_id', 'product_id'], how='left')
#deleting reorder_uxp
del reorder_uxp
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio
0,1,196,10,0.900000
1,1,10258,9,0.888889
2,1,10326,1,0.000000
3,1,12427,10,0.900000
4,1,13032,3,0.666667


In [ ]:
#merging users df into uxp
#the dataset includes samples of all eval_set (prior/train/test) data.
data = uxp.merge(users, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,196,10,0.900000,10,5.9,4
1,1,10258,9,0.888889,10,5.9,4
2,1,10326,1,0.000000,10,5.9,4
3,1,12427,10,0.900000,10,5.9,4
4,1,13032,3,0.666667,10,5.9,4


In [ ]:
#merging products df into data
data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
0,1,196,10,0.900000,10,5.9,4,35791,0.776480
1,1,10258,9,0.888889,10,5.9,4,1946,0.713772
2,1,10326,1,0.000000,10,5.9,4,5526,0.652009
3,1,12427,10,0.900000,10,5.9,4,6476,0.740735
4,1,13032,3,0.666667,10,5.9,4,3751,0.657158


In [ ]:
del [users, prd, uxp]

In [ ]:
data.shape 

(13307953, 9)

In [ ]:
orders = pd.read_csv('../../data/orders.csv')

In [ ]:
orders.eval_set.value_counts()

train    131209
test      75000
Name: eval_set, dtype: int64

In [ ]:
orders.head()

,Unnamed: 0,user_id,eval_set,order_id
0,10,1,train,1187899
1,25,2,train,1492625
2,38,3,test,2774568
3,44,4,test,329954
4,49,5,train,2196797


In [ ]:
orders.eval_set.unique()

array(['train', 'test'], dtype=object)

In [ ]:
#keeping only the train and test from eval_set orders dataframe.

#order_products_prior dataset was too big to work with.
#order_products_train(orders.eval_set=train) df was used instead, and enhanaced with FE.
#Use order.eval_set=='prior' to train df to avoid model overfitting.

order_future = orders.loc[((orders.eval_set == 'train') | (orders.eval_set == 'test')), ['user_id', 'eval_set', 'order_id']]
order_future.head()

,user_id,eval_set,order_id
0,1,train,1187899
1,2,train,1492625
2,3,test,2774568
3,4,test,329954
4,5,train,2196797


In [ ]:
order_future.eval_set.unique()

array(['train', 'test'], dtype=object)

In [ ]:
#merging the order_future with the data.
data = order_future.merge(data, on='user_id', how='left', sort='')
data.head()

,user_id,eval_set,order_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
0,1,train,1187899,196,10,0.900000,10,5.9,4,35791,0.776480
1,1,train,1187899,10258,9,0.888889,10,5.9,4,1946,0.713772
2,1,train,1187899,10326,1,0.000000,10,5.9,4,5526,0.652009
3,1,train,1187899,12427,10,0.900000,10,5.9,4,6476,0.740735
4,1,train,1187899,13032,3,0.666667,10,5.9,4,3751,0.657158


In [ ]:
#preparing the train df.
data_train = data.loc[(data.eval_set == 'train')]
data_train.head()

,user_id,eval_set,order_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
0,1,train,1187899,196,10,0.900000,10,5.9,4,35791,0.776480
1,1,train,1187899,10258,9,0.888889,10,5.9,4,1946,0.713772
2,1,train,1187899,10326,1,0.000000,10,5.9,4,5526,0.652009
3,1,train,1187899,12427,10,0.900000,10,5.9,4,6476,0.740735
4,1,train,1187899,13032,3,0.666667,10,5.9,4,3751,0.657158


In [ ]:
#merging the information from the order_products_train df into the data_train.
data_train = data_train.merge(order_products_train[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
data_train.head()

,user_id,eval_set,order_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio,reordered
0,1,train,1187899,196,10,0.900000,10,5.9,4,35791,0.776480,1.0
1,1,train,1187899,10258,9,0.888889,10,5.9,4,1946,0.713772,1.0
2,1,train,1187899,10326,1,0.000000,10,5.9,4,5526,0.652009,NaN
3,1,train,1187899,12427,10,0.900000,10,5.9,4,6476,0.740735,NaN
4,1,train,1187899,13032,3,0.666667,10,5.9,4,3751,0.657158,1.0


In [ ]:
#filling the NAN values
data_train.reordered.fillna(0, inplace=True)

In [ ]:
#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [ ]:
data_train.head()

uxp_times_bought  uxp_reordered_ratio  u_num_of_orders  \
user_id product_id                                                           
1       196                       10             0.900000               10   
        10258                      9             0.888889               10   
        10326                      1             0.000000               10   
        12427                     10             0.900000               10   
        13032                      3             0.666667               10   

                    u_avg_prd  dow_most_orders_u  prd_count_p  \
user_id product_id                                              
1       196               5.9                  4        35791   
        10258             5.9                  4         1946   
        10326             5.9                  4         5526   
        12427             5.9                  4         6476   
        13032             5.9                  4         3751   

                    p_reordered_ratio  reordered  
user_id product_id                                
1       196                  0.776480        1.0  
        10258                0.713772        1.0  
        10326                0.652009        0.0  
        12427                0.740735        0.0  
        13032                0.657158        1.0

In [ ]:
data_train.isnull().sum()

uxp_times_bought       0
uxp_reordered_ratio    0
u_num_of_orders        0
u_avg_prd              0
dow_most_orders_u      0
prd_count_p            0
p_reordered_ratio      0
reordered              0
dtype: int64

In [ ]:
#preparing the test dataset.
data_test = data[data.eval_set == 'test']
data_test.head()

,user_id,eval_set,order_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
120,3,test,2774568,248,1,0.000000,12,7.333333,0,6371,0.400251
121,3,test,2774568,1005,1,0.000000,12,7.333333,0,463,0.440605
122,3,test,2774568,1819,3,0.666667,12,7.333333,0,2424,0.492162
123,3,test,2774568,7503,1,0.000000,12,7.333333,0,12474,0.553551
124,3,test,2774568,8021,1,0.000000,12,7.333333,0,27864,0.591157


In [ ]:
data_test.isna().sum()

user_id                0
eval_set               0
order_id               0
product_id             0
uxp_times_bought       0
uxp_reordered_ratio    0
u_num_of_orders        0
u_avg_prd              0
dow_most_orders_u      0
prd_count_p            0
p_reordered_ratio      0
dtype: int64

In [ ]:
#deleting unwanted columns
data_test.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [ ]:
#setting user_id and product_id as index.
data_test = data_test.set_index(['user_id', 'product_id'])

In [ ]:
data_test.head()

uxp_times_bought  uxp_reordered_ratio  u_num_of_orders  \
user_id product_id                                                           
3       248                        1             0.000000               12   
        1005                       1             0.000000               12   
        1819                       3             0.666667               12   
        7503                       1             0.000000               12   
        8021                       1             0.000000               12   

                    u_avg_prd  dow_most_orders_u  prd_count_p  \
user_id product_id                                              
3       248          7.333333                  0         6371   
        1005         7.333333                  0          463   
        1819         7.333333                  0         2424   
        7503         7.333333                  0        12474   
        8021         7.333333                  0        27864   

                    p_reordered_ratio  
user_id product_id                     
3       248                  0.400251  
        1005                 0.440605  
        1819                 0.492162  
        7503                 0.553551  
        8021                 0.591157

In [ ]:
#deleting unwanted df
del [df, order_products_train, orders, order_future, data]

In [ ]:
#resetting index
data_train.reset_index(inplace=True)
data_test.reset_index(inplace=True)

In [ ]:
data_train.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio,reordered
0,1,196,10,0.900000,10,5.9,4,35791,0.776480,1.0
1,1,10258,9,0.888889,10,5.9,4,1946,0.713772,1.0
2,1,10326,1,0.000000,10,5.9,4,5526,0.652009,0.0
3,1,12427,10,0.900000,10,5.9,4,6476,0.740735,0.0
4,1,13032,3,0.666667,10,5.9,4,3751,0.657158,1.0


In [ ]:
data_train.to_csv('data_train.csv')
data_test.to_csv('data_test.csv')

# Save preprocessed data in new csv

In [ ]:
'''# save the data to a new csv file
#datapath = '../../data'
#save_file(df, 'processed.csv', datapath)

df.to_csv('processed_data.csv',index=False)'''